In [1]:
from PIL import Image
import numpy as np

def create_circular_mask(height, width, center=[964, 1110], radius=80):
    Y, X = np.ogrid[:height, :width]
    dist_from_center = np.sqrt((Y - center[0])**2 + (X - center[1])**2)
    mask = dist_from_center <= radius
    return mask

def convolve3d(image, kernel):
    output = np.zeros_like(image)
    image_padded = np.zeros((image.shape[0] + kernel.shape[0] - 1, image.shape[1] + kernel.shape[1] - 1, image.shape[2]))
    image_padded[kernel.shape[0] - 2:-1:, kernel.shape[1] - 2:-1:, :] = image
    image_padded[0, 0, :] = image[0, 0, :]
    image_padded[-1, -1, :] = image[-1, -1, :]
    for x in range(image.shape[1]):
        for y in range(image.shape[0]):
            for z in range(image.shape[2]):
                summation = (kernel * image_padded[y: y + kernel.shape[0], x: x + kernel.shape[1], z]).sum()
                output[y, x, z] = summation
    if image.shape[2] == 4:
        output[:, :, 3] = image[:, :, 3]
    return output

# Gaussian blur kernel
gaussian_blur_kernel = np.array([[1,  4,  6,  4, 1],
                                [4, 16, 24, 16, 4],
                                [6, 24, 36, 24, 6],
                                [4, 16, 24, 16, 4],
                                [1,  4,  6,  4, 1]]) / 256

input_file_name = "roi.jpg"
logo_file_name = "logo.png"

input_image = np.array(Image.open(input_file_name))
input_image = convolve3d(input_image, gaussian_blur_kernel)

# Create circular mask
circular_mask = create_circular_mask(input_image.shape[0], input_image.shape[1])

# Apply circular mask to the image
masked_image = input_image.copy()
masked_image[~circular_mask] = 0

# Shift and replace pixels
row_shift = -25
column_shift = -670
for i in range(input_image.shape[0]):
    for j in range(input_image.shape[1]):
        if masked_image[i, j, :].all() != 0:
            masked_image[i + row_shift, j + column_shift, :] = masked_image[i, j, :]
            masked_image[i, j, :] = 0
            input_image[i + row_shift, j + column_shift, :] = 0

# Combine the modified image and the masked image using bitwise OR
input_image = np.bitwise_or(input_image, masked_image)

# Save the final image
output_pil_img = Image.fromarray(input_image.astype(np.uint8))
output_pil_img.save('moved.jpeg')